In [1]:
%matplotlib inline

# 中文分詞(cntk)

In [2]:
import matplotlib
import matplotlib.pyplot as plt
from scipy import ndimage
import pylab
import PIL  
from PIL import Image
import os
import pickle
import codecs
import glob
import math
import cv2
import builtins
import string
import re
import cntk as C
from cntk.ops import *
from cntk.layers import *
from cntk.initializer import *
from cntk.logging import *
from cntk.train import *
from cntk.learners import *
from cntk.losses import *
from cntk.metrics import *
from cntk.device import *
import random

# 是否使用GPU
is_gpu = True
if is_gpu:
    try_set_default_device(gpu(0))
else:
    try_set_default_device(cpu())

In [3]:
def str_full_to_half(in_str):
    out_str = []
    for char in in_str:
        inside_code = ord(char)
        if inside_code == 0x3000 or inside_code == 12288 or char==string.whitespace: # 全形空格直接轉換
             out_str.append(' ')
        elif inside_code >= 65281 and inside_code <= 65374:
            inside_code -= 0xfee0
            out_str.append(chr(inside_code))
        else:
            out_str.append(char)
        
    return ''.join(out_str)

In [4]:
as_train=codecs.open('../Data/ex12_train/as_training.utf8',encoding='utf-8-sig').read()
cityu_train=codecs.open('../Data/ex12_train/cityu_training.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_train=as_train.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_train=cityu_train.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data=as_train+'\r\n'+cityu_train #把兩個語料合併
data=data.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data=str_full_to_half(data) #把所有全形轉半形

raw_data_train=data.split('\r\n')#分行

raw_data_train=[row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_train] #移除分行字元

print(raw_data_train[:20])



['時間|:', '三月|十日|(|星期四|)|上午|十時|。', '地點|:', '學術|活動|中心|一樓|簡報室|。', '主講|:', '民族所|所長|莊英章|先生|。', '講題|:', '閩|、|台|漢人|社會|研究|的|若干|考察|。', '李|院長|於|二月|二十六日|至|三月|十五日|赴|美|訪問|,', '期間|將|與|在|美|院士|商討|院務|,', '與|美國|大學|聯繫|商討|長期|合作|事宜|,', '並|辦理|加州|大學|退休|等|手續|。', '出國|期間|院務|由|羅|副院長|代行|。', '總辦事處|秘書組|主任|戴政|先生|請辭|獲准|,', '所|遺|職務|自|三月|一日|起|由|近代史|研究所|研究員|陶英惠|先生|兼任|。', '植物|研究所|所長|周昌弘|先生|當選|第三世界|科學院|(|The|Third|World|Academy|of|Sciences|,', '簡稱|TWAS|)|院士|。', 'TWAS|係|一九八三年|由|Prof|Adbus|Salam|(|巴基斯坦籍|,', '曾|獲|諾貝爾獎|)|發起|成立|,', '會員|遍佈|63|個|國家|,']


In [5]:
as_test=codecs.open('../Data/ex12_train/as_testing_gold.utf8',encoding='utf-8-sig').read()
cityu_test=codecs.open('../Data/ex12_train/cityu_test_gold.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_test=as_test.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_test=cityu_test.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data_test=as_test+'\r\n'+cityu_test #把兩個語料合併
data_test=data_test.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data_test=str_full_to_half(data_test) #把所有全形轉半形

raw_data_test=data_test.split('\r\n')#分行

raw_data_test=[row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_test] #移除分行字元

print(raw_data_test[:20])


['許多|社區|長青|學苑|多|開設|有|書法|、|插花|、|土風舞班|,', '文山區|長青|學苑|則|有|個|十分|特別|的|「|英文|歌唱班|」|,', '成員|年齡|均|超過|六十|歲|,', '這|群|白髮蒼蒼|,', '爺爺|、|奶奶級|的|學員|唱起|英文|歌|來|字正腔圓|,', '有模有樣|。', '對|他們|來說|,', '能|與|一|群|志同道合|的|朋友|共同|回味|年少|時期|流行|的|歌曲|,', '才|是|參加|英文|歌唱班|最|大|樂趣|。', '長青|學苑|英文|歌唱班|昨日|在|社教館|舉行|「|英文|懷念|金曲|演唱會|」|,', '曲目|包括|「|大江東去|」|、|「|月河|」|、|「|真善美|」|等|大眾|耳熟能詳|的|英文|歌曲|。', '難得|公開|演唱|,', '這些|有|著|豐富|人生|閱歷|的|學員|絲毫|不|覺得|緊張|怯場|,', '只|見|台|上|唱|得|盡興|,', '台|下|不少|聽眾|也|一時|技癢|跟|著|唱和|起來|。', '長青|學苑|英文|歌唱班|成立|至今|已|兩|年|,', '目前|成員|約|廿五|人|,', '年齡|都|在|六十|歲|以上|,', '其中|以|軍公教|退休|人員|居多|,', '並|有|現任|大學|教授|,']


In [6]:
vocabs=sorted(set(list(''.join(raw_data_train))))
vocabs.remove('|')
vocabs.insert(0,'/Unknow')

print(len(vocabs))
print(vocabs)

6295
['/Unknow', ' ', '!', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '°', '·', '×', 'ˊ', 'ˋ', '˙', 'Α', 'Β', 'Γ', 'Ε', 'Η', 'Ι', 'Κ', 'Μ', 'Ν', 'Ο', 'Ρ', 'Τ', 'α', 'β', 'λ', 'μ', 'ω', '—', '…', '‧', '※', '℃', 'Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ', 'Ⅴ', 'Ⅹ', '↑', '→', '∕', '∥', '─', '│', '╳', '■', '□', '▲', '◆', '◇', '○', '◎', '●', '◢', '★', '☆', '、', '。', '〇', '〈', '〉', '《', '》', '「', '」', '『', '』', '【', '】', '〔', '〕', 'ㄅ', 'ㄆ', 'ㄇ', 'ㄈ', 'ㄉ', 'ㄊ', 'ㄋ', 'ㄌ', 'ㄍ', 'ㄎ', 'ㄏ', 'ㄐ', 'ㄑ', 'ㄒ', 'ㄓ', 'ㄔ', 'ㄕ', 'ㄖ', 'ㄗ', 'ㄘ', 'ㄙ', 'ㄚ', 'ㄛ', 'ㄜ', 'ㄝ', 'ㄞ', 'ㄟ', 'ㄠ', 'ㄡ', 'ㄢ', 'ㄣ', 'ㄤ', 'ㄥ', 'ㄦ', 'ㄧ', 'ㄨ', 'ㄩ', '㎜', '㎝', '㎡', '㏄', '㗎', '䠷

In [7]:
char_to_index=dict((w, i) for i, w in enumerate(vocabs))
index_to_char=dict((i, w) for i, w in enumerate(vocabs))

print(char_to_index)

{'餾': 5985, '椗': 2598, '電': 5833, 'Ρ': 104, '倉': 397, '僇': 461, '嫦': 1346, '脖': 4343, '塾': 1161, '牡': 3282, '●': 136, '肄': 4291, '法': 2863, '式': 1687, '隄': 5784, '啖': 884, '瞭': 3692, '猜': 3320, '氛': 2785, '印': 695, '搪': 2167, '耦': 4267, '恙': 1808, '散': 2290, '掮': 2130, '菱': 4576, 'ㄣ': 185, '燎': 3234, '緲': 4141, '梗': 2558, '芥': 4467, '訶': 4968, '表': 4850, '及': 717, '膜': 4385, '谷': 5095, '琶': 3407, '咎': 812, '峻': 1525, '布': 1593, '卿': 704, '挂': 2065, '江': 2814, '禱': 3826, '痄': 3529, '既': 2333, '鵠': 6166, '撈': 2202, '綿': 4125, '苜': 4492, '鎬': 5667, '欲': 2713, '煒': 3195, '嶮': 1563, '孻': 1391, '躪': 5285, 'ㄗ': 173, '兄': 497, '蠻': 4837, '糰': 4050, '做': 431, '鯢': 6107, '栲': 2530, '儼': 493, '支': 2270, '訓': 4954, '瘐': 3558, '璀': 3429, '妗': 1259, '靛': 5862, '九': 239, '剌': 595, '捉': 2084, '幀': 1612, '珣': 3379, '子': 1367, '業': 2616, '迸': 5367, '狒': 3306, '曹': 2432, '鷺': 6201, '䠷': 197, '鋁': 5607, '兆': 499, '炸': 3164, '蝟': 4787, '盛': 3622, '忡': 1769, '虯': 4730, '寞': 1434, '歎': 2722, '骼': 6057, '站': 

In [8]:
idx_train=0
idxs_train=np.arange(len(raw_data_train))
np.random.shuffle(idxs_train)

idx_test=0
idxs_test=np.arange(len(raw_data_test))
np.random.shuffle(idxs_test)

def get_next_minibatch(minibatch_size,is_train=True):
    global idx_train,idxs_train,raw_data_train,idx_test,idxs_test,raw_data_test
    features=[]
    labels=[]
    groundtruths=[]
    idx=idx_train
    idxs=idxs_train
    raw_data=raw_data_train
    if is_train==False:
        idx=idx_test
        idxs=idxs_test
        raw_data=raw_data_test
    np.random.shuffle(raw_data)
    
    while len(features)<minibatch_size:
        features_seq=[]
        labels_seq=[]
        groundtruth_seq=[]
        words=raw_data[idxs[idx]].split('|') #轉換成分詞後的詞清單
        #BMES=>[0,1,2,3]
        for word in words:
            for i in range(len(word)):
                feature_arr=np.zeros(len(vocabs),dtype=np.float32)
                label_arr=np.zeros(4,dtype=np.float32)
                #如果在字典中則取出其索引
                if word[i] in char_to_index:
                    feature_arr[char_to_index[word[i]]]=1
                #否則定為未知
                else:
                    feature_arr[char_to_index['/Unknow']]=1
                
                #轉換為BMES
                if len(word)==1 and i==0: #S 自己就是一個單詞
                    label_arr[3]=np.random.choice(np.arange(0.8, 1.2, 0.01))
                elif i==0:
                    label_arr[0]=np.random.choice(np.arange(0.8, 1.2, 0.01))  #B 是一個詞的開始
                elif i==len(word)-1:  #B 是一個詞的開始
                    label_arr[2]=np.random.choice(np.arange(0.8, 1.2, 0.01))
                else:
                    label_arr[1]=np.random.choice(np.arange(0.8, 1.2, 0.01))
                #測試集不進行label smoothing
                if is_train==False:
                    label_arr[label_arr>0]=1
                    
                features_seq.append(feature_arr)
                labels_seq.append(label_arr)
            groundtruth_seq.append(word)
        idx+=1
        
        if idx>len(idxs):
            idx=0
            np.random.shuffle(idxs)
            
        features.append(features_seq)
        labels.append(labels_seq)
        groundtruths.append(groundtruth_seq)
        
    idx_train=idx
    if is_train==False:
        idx_test=idx
    return features,labels,groundtruths
        
x_feature,y_label,ground_truths=get_next_minibatch(2)
print(x_feature)
print(y_label)
print(ground_truths)

[[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)], [array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)]]
[[array([0.  , 0.  , 0.  , 1.16], dtype=float32), array([1.18, 0.  , 0.  , 0.  ], dtype=float32), array([0.  , 1.03, 0.  , 0.  ], dtype=float32), array([0.  , 0.  , 1.02, 0.  ], dtype=float32), array([0., 0., 0., 1.], dtype=float32), array([0.  , 0.  , 0.  , 0.84], dtype=float32), array([0.  , 0.  , 0.  , 0.93], dtype=float32)], [array([0.95, 0.  , 0.  , 0.  ], dtype=float32), array([0. , 0. , 0.8, 0. ], dtyp

In [ ]:
#定義序列軸
input_seq_axis = Axis('inputAxis')

#定義輸入變數
input_sequence = sequence.input_variable(shape=len(vocabs), sequence_axis=input_seq_axis)
label_sequence = sequence.input_variable(shape=4, sequence_axis=input_seq_axis) #BMES四種


#雙向遞迴
def BiRecurrence(fwd, bwd):
    F = C.layers.Recurrence(fwd)
    G = C.layers.Recurrence(bwd, go_backwards=True)
    x = C.placeholder()
    apply_x = C.splice(F(x), G(x))
    return apply_x 

def create_model(num_layers=2,hidden_dim=512):
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(256),
            For(range(num_layers), lambda: 
                Sequential([Stabilizer(), 
                            BiRecurrence(LSTM(hidden_dim//2),LSTM(hidden_dim//2)),
                            BatchNormalization()])),
            Dropout(0.5),
            C.layers.Dense(4)
        ])
 

In [ ]:
learning_rate=0.0001
minibatch_size=24
num_epochs=20

#宣告模型結構
rnn=create_model()
z=rnn(input_sequence)
if not os.path.exists('Models'):
    os.mkdir('Models')
    print("Directory Models Created ")
    
if os.path.exists("Models/word_segment_cntk.lstm"):
    model=Function.load("Models/word_segment_cntk.lstm")
    z = model(input_sequence)
    print('model recovered!')

#定義進行訓練的損失函數以及錯誤率計算
loss= cross_entropy_with_softmax(z,label_sequence)#+0.1*focal_loss(z,label_sequence)
errs = classification_error(z, label_sequence)

# 列印模型參數
log_number_of_parameters(z);

# 定義訓練器

progress_printer = ProgressPrinter(freq=100, tag='Training', num_epochs=num_epochs)
learner = adam(z.parameters,
                lr=learning_rate_schedule([learning_rate], UnitType.sample, num_epochs),
                momentum=momentum_as_time_constant_schedule([minibatch_size / -math.log(0.95)], epoch_size=num_epochs),
                l2_regularization_weight=1e-3)
trainer = Trainer(z, (loss, errs), learner, progress_printer)
for epoch in range(num_epochs):
    mbs = 0
    progress_printer.update_with_trainer(trainer, with_metric=True)
    num_trained_samples = 0
    while mbs<2000:
        features, labels,ground_truths=get_next_minibatch(minibatch_size)
        #進行訓練
        trainer.train_minibatch({input_sequence: features, label_sequence: labels})
        if mbs%100==0 and mbs>0:
            z.save("Models/word_segment_cntk.lstm")
        if mbs%500==0 :
            features_test, labels_test,ground_truths_test=get_next_minibatch(minibatch_size,False)
            
            trainer.test_minibatch({input_sequence: features_test, label_sequence: labels_test})
            print('-----測試集驗證--------')
            trainer.summarize_test_progress()
            result=z(features_test)
            for i in range(3):
                answer='|'.join(ground_truths_test[i])+'|'
                pred=[]
                words=list(''.join(ground_truths_test[i]))
                for j in range(len(words)):
                    word=words[j]
                    onehot=np.argmax(result[i][j])
                    if onehot>=2:
                        pred.append(word+'|')
                    else:
                        pred.append(word)
                
                pred=''.join(pred)  
                print('predict:'+pred)
                print('answer :'+answer)
            print('-----------------')
            

        if mbs%1000==0 and mbs>0:
            learning_rate*=0.75

        mbs += 1
    #回報每個epoch訓練進度以及相關指標
    trainer.summarize_training_progress()
   
    

model recovered!
Training 4241158 parameters in 21 parameter tensors.
Learning rate per 1 samples: 0.0001
-----測試集驗證--------
Finished Evaluation [1]: Minibatch[1-1]: metric = 11.26% * 399;
predict:並|請|詳填|以下|聯絡|資料|回傳|。|
answer :並|請|詳填|以下|聯絡|資料|回傳|。|
predict:其中|還|有|人|是|被|女朋友|拉來|的|,|
answer :其中|還|有|人|是|被|女朋友|拉來|的|,|
predict:凡|在|戰場|上|擄獲|的|印章|必須|上繳|;|
answer :凡|在|戰場|上|擄獲|的|印章|必須|上繳|;|
-----------------
 Minibatch[   1- 100]: loss = 0.216332 * 35445, metric = 8.22% * 35445;
 Minibatch[ 101- 200]: loss = 0.225088 * 33786, metric = 8.42% * 33786;
 Minibatch[ 201- 300]: loss = 0.227613 * 34861, metric = 8.64% * 34861;
 Minibatch[ 301- 400]: loss = 0.229985 * 33762, metric = 8.81% * 33762;
 Minibatch[ 401- 500]: loss = 0.229033 * 34769, metric = 8.41% * 34769;
-----測試集驗證--------
Finished Evaluation [2]: Minibatch[1-1]: metric = 13.59% * 401;
predict:收視|佳|,|
answer :收視|佳|,|
predict:「|伴|著|她|一|路|挺進|的|,|是|高達|86.7|分貝|的|尖叫|」|,|「|和|戴芬波特|搶|七時|,|叫|得像是|非洲|猿猴|。|」|
answer :「|伴著|她|一路|挺進|的|,|是|高|達|86.7|分貝|的|尖